In [1]:
import pandas as pd
import numpy as np
import os
print("pandas version : ",pd.__version__)
print("numpy version : ", np.__version__)

pandas version :  1.0.5
numpy version :  1.19.1


In [2]:
data = pd.read_csv("data/new_performance_ver3.csv")
names = data.columns
for name in names:
    if "Unnamed" in name:
        data.pop(name)
data.head()

,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액,date,week,time,요일,IsHoliday
0,2019-01-01 06:00:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,2099000.0,2019-01-01,1,06:00:00,1.0,1.0
1,2019-01-01 06:00:00,20.0,100346,201079,테이트 여성 셀린니트3종,의류,39900,4371000.0,2019-01-01,1,06:00:00,1.0,1.0
2,2019-01-01 06:20:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,3262000.0,2019-01-01,1,06:20:00,1.0,1.0
3,2019-01-01 06:20:00,20.0,100346,201079,테이트 여성 셀린니트3종,의류,39900,6955000.0,2019-01-01,1,06:20:00,1.0,1.0
4,2019-01-01 06:40:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,6672000.0,2019-01-01,1,06:40:00,1.0,1.0


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35379 entries, 0 to 35378
Data columns (total 13 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   방송일시       35379 non-null  object 
 1   노출(분)      35379 non-null  float64
 2   마더코드       35379 non-null  int64  
 3   상품코드       35379 non-null  int64  
 4   상품명        35379 non-null  object 
 5   상품군        35379 non-null  object 
 6   판매단가       35379 non-null  int64  
 7   취급액        35379 non-null  float64
 8   date       35379 non-null  object 
 9   week       35379 non-null  int64  
 10  time       35379 non-null  object 
 11  요일         1324 non-null   float64
 12  IsHoliday  35379 non-null  float64
dtypes: float64(4), int64(4), object(5)
memory usage: 3.5+ MB


요일에는 null값이 왜이렇게 많을까요..?

In [4]:
data.isna().sum()

방송일시             0
노출(분)            0
마더코드             0
상품코드             0
상품명              0
상품군              0
판매단가             0
취급액              0
date             0
week             0
time             0
요일           34055
IsHoliday        0
dtype: int64

In [12]:
weather = pd.read_csv("월별인구가중합날씨.csv")
weather.head()

,time,TEMP,HUM
0,2019-01-01 01:00,-5.157666,53.621521
1,2019-01-01 02:00,-5.577352,56.876710
2,2019-01-01 03:00,-5.922292,58.845987
3,2019-01-01 04:00,-6.223195,60.195692
4,2019-01-01 05:00,-6.449101,61.929722


In [13]:
len(weather)

13080

In [7]:
def hoursplit(data):
    return data[:2]
def minsplit(data):
    return data[3:5]

In [8]:
data1 = data.copy()
data1['hour'] = data['time'].apply(lambda x: hoursplit(x))
data1['min'] = data['time'].apply(lambda x: minsplit(x))
data1.head(2)

,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액,date,week,time,요일,IsHoliday,hour,min
0,2019-01-01 06:00:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,2099000.0,2019-01-01,1,06:00:00,1.0,1.0,06,00
1,2019-01-01 06:00:00,20.0,100346,201079,테이트 여성 셀린니트3종,의류,39900,4371000.0,2019-01-01,1,06:00:00,1.0,1.0,06,00


In [9]:
data1.isna().sum()

방송일시             0
노출(분)            0
마더코드             0
상품코드             0
상품명              0
상품군              0
판매단가             0
취급액              0
date             0
week             0
time             0
요일           34055
IsHoliday        0
hour             0
min              0
dtype: int64

In [10]:
from datetime import datetime

## Merging

data1과 weather를 날짜와 '시'로 merge해주기 위해 weather의 time컬럼을 date와 hour로 나누어 주는 작업을 진행했습니다.

- datetime모듈의 strptime과 strftime을 이중으로 lamda함수에 넣어주었습니다.
- 컬럼 명을 data1과 동일하게 date, hour로 설정하였습니다.

In [14]:
weather['real_date'] = weather['time'].apply(lambda x: datetime.strptime(x,
                                                                   '%Y-%m-%d %H:%M').strftime("%Y-%m-%d"))
weather['hour'] = weather['time'].apply(lambda x: datetime.strptime(x,
                                                                   '%Y-%m-%d %H:%M').strftime("%H"))
weather = weather.drop(['time'], axis=1)

weather.head()

,TEMP,HUM,real_date,hour
0,-5.157666,53.621521,2019-01-01,01
1,-5.577352,56.876710,2019-01-01,02
2,-5.922292,58.845987,2019-01-01,03
3,-6.223195,60.195692,2019-01-01,04
4,-6.449101,61.929722,2019-01-01,05


동일한 hour를 가진 행에는 동일한 날씨 정보가 삽입 되었습니다. 근데 date는 0시는 전날로 들어감..!! 1월 정보랑 방송 일시 기준 날짜를 다시 넣어야 할듯!!

In [15]:
data1['real_date'] = data1['방송일시'].apply(lambda x: datetime.strptime(x,
                                                                   '%Y-%m-%d %H:%M:%S').strftime("%Y-%m-%d"))
data1.head(2)

,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액,date,week,time,요일,IsHoliday,hour,min,real_date
0,2019-01-01 06:00:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,2099000.0,2019-01-01,1,06:00:00,1.0,1.0,06,00,2019-01-01
1,2019-01-01 06:00:00,20.0,100346,201079,테이트 여성 셀린니트3종,의류,39900,4371000.0,2019-01-01,1,06:00:00,1.0,1.0,06,00,2019-01-01


- date와 상품코드 이용해서 단일 방송 구하기(groupby)

In [18]:
# del data1['요일']
# # null 값이 너무 많아서 일단 지워주고 진행하겠습니다.

In [22]:
df = pd.merge(data1, weather, on = ['real_date', 'hour'], how = 'outer')
df = df.loc[df['real_date']<='2020-01-01'].dropna()
display(df.head(2))
display(df.tail(2))

,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액,date,week,time,IsHoliday,hour,min,real_date,TEMP,HUM
0,2019-01-01 06:00:00,20.0,100346.0,201072.0,테이트 남성 셀린니트3종,의류,39900.0,2099000.0,2019-01-01,1.0,06:00:00,1.0,06,00,2019-01-01,-6.576974,63.524958
1,2019-01-01 06:00:00,20.0,100346.0,201079.0,테이트 여성 셀린니트3종,의류,39900.0,4371000.0,2019-01-01,1.0,06:00:00,1.0,06,00,2019-01-01,-6.576974,63.524958


,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액,date,week,time,IsHoliday,hour,min,real_date,TEMP,HUM
35282,2019-12-31 00:40:00,20.0,100320.0,201300.0,올바로 문어발 샤워기 1+1,생활용품,54900.0,23852000.0,2019-12-30,0.0,00:40:00,0.0,00,40,2019-12-31,-1.732468,59.667646
35283,2019-12-31 00:40:00,20.0,100320.0,201301.0,올바로 문어발 샤워기 1세트,생활용품,28900.0,9273000.0,2019-12-30,0.0,00:40:00,0.0,00,40,2019-12-31,-1.732468,59.667646


Merge 과정에서 1월 1일 정보가 drop되어 2020.1 ~ 2020.6의 날씨 정보를 업데이트 해야할 것 같습니다. 200개 정도가 na값으로 처리된 것으로 보입니다.

In [23]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 35284 entries, 0 to 35283
Data columns (total 17 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   방송일시       35284 non-null  object 
 1   노출(분)      35284 non-null  float64
 2   마더코드       35284 non-null  float64
 3   상품코드       35284 non-null  float64
 4   상품명        35284 non-null  object 
 5   상품군        35284 non-null  object 
 6   판매단가       35284 non-null  float64
 7   취급액        35284 non-null  float64
 8   date       35284 non-null  object 
 9   week       35284 non-null  float64
 10  time       35284 non-null  object 
 11  IsHoliday  35284 non-null  float64
 12  hour       35284 non-null  object 
 13  min        35284 non-null  object 
 14  real_date  35284 non-null  object 
 15  TEMP       35284 non-null  float64
 16  HUM        35284 non-null  float64
dtypes: float64(9), object(8)
memory usage: 4.8+ MB


## 방송 초/중/후반 정보 삽입

- 방송일시와 상품코드 사용
    - 동일 방송 내에서 여러 상품을 판매하는 경우가 있음
- pandas의 quantile()함수를 통해 데이터가 위치한 곳을 찾아 정보를 삽입

In [36]:
def get_stage(q):
    if q <= 0.33:
        return '초반'
    elif 0.33 < q <= 0.66:
        return '중반'
    return '후반'

In [90]:
sample = df[['방송일시', 'date', '상품코드', '노출(분)']]
sample['cumcount'] = df.groupby(['date', '상품코드'])['노출(분)'].cumcount()+1
sample.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,방송일시,date,상품코드,노출(분),cumcount
0,2019-01-01 06:00:00,2019-01-01,201072.0,20.0,1
1,2019-01-01 06:00:00,2019-01-01,201079.0,20.0,1
2,2019-01-01 06:20:00,2019-01-01,201072.0,20.0,2
3,2019-01-01 06:20:00,2019-01-01,201079.0,20.0,2
4,2019-01-01 06:40:00,2019-01-01,201072.0,20.0,3


In [91]:
sample2 = df.groupby(['date', '상품코드'])['노출(분)'].count().reset_index()
sample2.rename({'노출(분)' : 'count'}, axis = 1, inplace = True)

sample2.head()

,date,상품코드,count
0,2019-01-01,200197.0,3
1,2019-01-01,200236.0,3
2,2019-01-01,200465.0,3
3,2019-01-01,200533.0,3
4,2019-01-01,200652.0,3


In [92]:
sample = sample.merge(sample2, on = ['date', '상품코드'])
sample.head()

,방송일시,date,상품코드,노출(분),cumcount,count
0,2019-01-01 06:00:00,2019-01-01,201072.0,20.0,1,3
1,2019-01-01 06:20:00,2019-01-01,201072.0,20.0,2,3
2,2019-01-01 06:40:00,2019-01-01,201072.0,20.0,3,3
3,2019-01-01 06:00:00,2019-01-01,201079.0,20.0,1,3
4,2019-01-01 06:20:00,2019-01-01,201079.0,20.0,2,3


In [93]:
sample['p'] = sample['cumcount']/sample['count']
sample.head()

,방송일시,date,상품코드,노출(분),cumcount,count,p
0,2019-01-01 06:00:00,2019-01-01,201072.0,20.0,1,3,0.333333
1,2019-01-01 06:20:00,2019-01-01,201072.0,20.0,2,3,0.666667
2,2019-01-01 06:40:00,2019-01-01,201072.0,20.0,3,3,1.000000
3,2019-01-01 06:00:00,2019-01-01,201079.0,20.0,1,3,0.333333
4,2019-01-01 06:20:00,2019-01-01,201079.0,20.0,2,3,0.666667


## stage, stage_advanced

count가 2혹은 1인 경우 각각에 대해 ['early', 'late'], 'short'로 바꾸어주고 3이상인 경우 stage함수를 사용해서 1/3로 나눈 분기점에 대해서 나누어진다

In [94]:
def stage(x):
    if x < 0.34:
        return 'early'
    elif 0.34 <= x < 0.67:
        return 'mid'
    return 'late'

def stage_advanced(x):
    if x['count'] == 1:
        return 'short'
    if x['count'] == 2:
        if x['p'] <= 0.5:
            return 'early'
        else:
            return 'late'
    else:
        return stage(x['p'])
            

In [96]:
sample['stage'] = sample.apply(stage_advanced, axis=1)

In [97]:
sample

,방송일시,date,상품코드,노출(분),cumcount,count,p,stage
0,2019-01-01 06:00:00,2019-01-01,201072.0,20.0,1,3,0.333333,early
1,2019-01-01 06:20:00,2019-01-01,201072.0,20.0,2,3,0.666667,mid
2,2019-01-01 06:40:00,2019-01-01,201072.0,20.0,3,3,1.000000,late
3,2019-01-01 06:00:00,2019-01-01,201079.0,20.0,1,3,0.333333,early
4,2019-01-01 06:20:00,2019-01-01,201079.0,20.0,2,3,0.666667,mid
...,...,...,...,...,...,...,...,...
35279,2019-12-31 00:00:00,2019-12-30,201906.0,20.0,3,3,1.000000,late
35280,2019-12-31 00:20:00,2019-12-30,201300.0,20.0,1,2,0.500000,early
35281,2019-12-31 00:40:00,2019-12-30,201300.0,20.0,2,2,1.000000,late
35282,2019-12-31 00:20:00,2019-12-30,201301.0,20.0,1,2,0.500000,early


In [98]:
sample[['방송일시', 'date', '상품코드', '노출(분)', 'stage']].to_csv("stage.csv", index=False)